In [4]:
import pandas as pd
import numpy as np
import glob
import os
import re, operator, warnings
warnings.filterwarnings('ignore')

In [5]:
#import the weather files for top 20 origin airports of 2018 obtained from:
# https://mesonet.agron.iastate.edu/ASOS/
path = '../data/raw/2017/'
flight_path = '../data/interim/'
all_files = glob.glob(os.path.join(path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent

df_from_each_file = (pd.read_csv(f, parse_dates=['valid']) for f in all_files)
concatenated_df = pd.concat(df_from_each_file)

In [6]:
weather_df = concatenated_df

In [7]:
weather_df.columns

Index(['station', 'valid', 'tmpf', 'dwpf', 'relh', 'drct', 'sknt', 'p01i',
       'alti', 'mslp', 'vsby', 'gust', 'skyc1', 'skyc2', 'skyc3', 'skyc4',
       'skyl1', 'skyl2', 'skyl3', 'skyl4', 'wxcodes', 'ice_accretion_1hr',
       'ice_accretion_3hr', 'ice_accretion_6hr', 'peak_wind_gust',
       'peak_wind_drct', 'peak_wind_time', 'feel', 'metar'],
      dtype='object')

In [8]:
weather_df = weather_df[['station','valid', 'tmpf','vsby','gust','sknt','p01i',
                         'ice_accretion_3hr','ice_accretion_6hr',]]

In [9]:
weather_df.columns

Index(['station', 'valid', 'tmpf', 'vsby', 'gust', 'sknt', 'p01i',
       'ice_accretion_3hr', 'ice_accretion_6hr'],
      dtype='object')

In [10]:
weather_df = weather_df.fillna(0)

In [11]:
weather_df['p01i'] = pd.to_numeric(weather_df.p01i, errors='coerce')

In [12]:
weather_df['valid_new'] = weather_df.valid.apply(lambda x: x.replace(minute=0))

In [13]:
df1 = pd.DataFrame(weather_df.groupby(['station','valid_new'])['tmpf','vsby'].min())
df2 = pd.DataFrame(weather_df.groupby(['station','valid_new'])['p01i'].max())
df3 = pd.DataFrame(weather_df.groupby(['station','valid_new'])['gust','sknt'].max())

In [14]:
df_test = pd.merge(df1, df2, how='left', left_index=True, right_index=True)

In [15]:
df_test = df_test.merge(df3, how='left', left_index=True, right_index=True)

In [16]:
df_test.head()

tmpf  vsby  p01i  gust  sknt
station valid_new                                        
ATL     2017-01-01 00:00:00   0.0   5.0  0.05  15.0  10.0
        2017-01-01 01:00:00   0.0   5.0  0.04  23.0  19.0
        2017-01-01 02:00:00   0.0   5.0  0.04   0.0   8.0
        2017-01-01 03:00:00   0.0   6.0  0.02   0.0   9.0
        2017-01-01 04:00:00   0.0   5.0  0.01   0.0   8.0

In [17]:
df_test.reset_index(inplace=True) 

In [18]:
df_test.head()

,station,valid_new,tmpf,vsby,p01i,gust,sknt
0,ATL,2017-01-01 00:00:00,0.0,5.0,0.05,15.0,10.0
1,ATL,2017-01-01 01:00:00,0.0,5.0,0.04,23.0,19.0
2,ATL,2017-01-01 02:00:00,0.0,5.0,0.04,0.0,8.0
3,ATL,2017-01-01 03:00:00,0.0,6.0,0.02,0.0,9.0
4,ATL,2017-01-01 04:00:00,0.0,5.0,0.01,0.0,8.0


In [19]:
weather_df_new = df_test.copy().fillna(0)

In [20]:
weather_df_new.head()

,station,valid_new,tmpf,vsby,p01i,gust,sknt
0,ATL,2017-01-01 00:00:00,0.0,5.0,0.05,15.0,10.0
1,ATL,2017-01-01 01:00:00,0.0,5.0,0.04,23.0,19.0
2,ATL,2017-01-01 02:00:00,0.0,5.0,0.04,0.0,8.0
3,ATL,2017-01-01 03:00:00,0.0,6.0,0.02,0.0,9.0
4,ATL,2017-01-01 04:00:00,0.0,5.0,0.01,0.0,8.0


In [21]:
#export raw, concoctanated weather file for 2017 top 20 airports
weather_df_new.to_csv(f'{path}/2017WeatherALL.csv', index = False, encoding = 'utf-8')

In [22]:
#import the 2017 Flight Data and clean it
df = pd.read_csv(f'{flight_path}/2017ALL.csv', parse_dates=['FlightDate'])


df = df.dropna(subset=['Tail_Number','DepTime','DepDelay','DepartureDelayGroups','TaxiOut',
                      'WheelsOff','WheelsOn','TaxiIn','ArrTime','ArrDelay','CRSElapsedTime',
                      'ActualElapsedTime','AirTime'])

df = df.fillna(0)

In [23]:
#Let's remove the features that will not be available for predicting flights
df = df[['Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate',
       'Reporting_Airline', 'Tail_Number', 'Flight_Number_Reporting_Airline',
       'Origin', 'OriginState', 'DestState', 'Dest', 'CRSDepTime', 'CRSArrTime', 'CRSElapsedTime',
       'ArrDelay', 'Distance', 'DistanceGroup',]]
df = df.reindex()

In [24]:
#since the DEP_TIME is presented as a float we need to convert it to a Timestamp
def convert_to_time (datatype):
    '''This method will take a float as input and returns a datetime.time object'''
    to_string = str(int(datatype))
    #check length of string and add leading zeros if lenght is less than 4
    while len(to_string) < 4:
        to_string = '0' + to_string
    #check if hour starts with 24 then change to 00
    if to_string[:2] == '24':
        to_string = '00' + to_string[2:]
    #obj = time.strptime(to_string, '%H%M')
    
    to_string = to_string[:2] + ':' + to_string[2:] 
    datetime = pd.to_datetime(to_string)
    datetime = pd.to_datetime(to_string, format='%H:%M')
    time = datetime.time()
    return time
    
    #datetime = time.strftime("%H:%M", obj)
    #return datetime
    
#We now need to create a new Timestamp column incorporating the information from FL_DATE and DEP_TIME
def combine_datetime (dataframe):
    '''This function will extract needed data from two columns and combine into a single Timestamp object'''
    year = dataframe[0].year
    month = dataframe[0].month
    day = dataframe[0].day
    hour = dataframe[1].hour
    minute = dataframe[1].minute
    second = dataframe[1].second
    
    datetime = pd.Timestamp(year, month, day, hour, minute, second)
    return datetime

In [25]:
combined = df[['FlightDate','CRSDepTime']]

In [26]:
#convert DEP_TIME to timestamp
combined['CRSDepTime'] = df.CRSDepTime.apply(convert_to_time)

In [27]:
#apply the combine_datetime function to optimized_df and create a new column 'FL_TIME'
df['FL_TIME'] = combined.apply(combine_datetime, axis=1)

In [28]:
df['FL_TIME_NEW'] = df.FL_TIME.apply(lambda x: x.replace(minute=0))

In [29]:
df.head()

,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,Origin,OriginState,DestState,Dest,CRSDepTime,CRSArrTime,CRSElapsedTime,ArrDelay,Distance,DistanceGroup,FL_TIME,FL_TIME_NEW
0,2,5,6,6,2017-05-06,WN,N7824A,4652,SJC,CA,CA,SAN,650,810,80.0,2.0,417.0,2,2017-05-06 06:50:00,2017-05-06 06:00:00
1,2,5,6,6,2017-05-06,WN,N8522P,4971,SJC,CA,CA,SAN,1710,1830,80.0,-7.0,417.0,2,2017-05-06 17:10:00,2017-05-06 17:00:00
2,2,5,6,6,2017-05-06,WN,N8617E,5113,SJC,CA,CA,SAN,1100,1220,80.0,0.0,417.0,2,2017-05-06 11:00:00,2017-05-06 11:00:00
3,2,5,6,6,2017-05-06,WN,N450WN,5150,SJC,CA,CA,SAN,2030,2150,80.0,1.0,417.0,2,2017-05-06 20:30:00,2017-05-06 20:00:00
4,2,5,6,6,2017-05-06,WN,N498WN,5711,SJC,CA,CA,SAN,1325,1445,80.0,-2.0,417.0,2,2017-05-06 13:25:00,2017-05-06 13:00:00


In [30]:
weather_df_new1 = weather_df_new.copy()

In [31]:
#remove rows will null values which is approximately 1.5%
null_columns = weather_df_new1.columns[weather_df_new1.isnull().any()]
weather_df_new1[null_columns].isnull().sum()

Series([], dtype: float64)

In [32]:
weather_df_new.head()

,station,valid_new,tmpf,vsby,p01i,gust,sknt
0,ATL,2017-01-01 00:00:00,0.0,5.0,0.05,15.0,10.0
1,ATL,2017-01-01 01:00:00,0.0,5.0,0.04,23.0,19.0
2,ATL,2017-01-01 02:00:00,0.0,5.0,0.04,0.0,8.0
3,ATL,2017-01-01 03:00:00,0.0,6.0,0.02,0.0,9.0
4,ATL,2017-01-01 04:00:00,0.0,5.0,0.01,0.0,8.0


In [33]:
#combine the station name and datetime for easier / faster merging with flight data
weather_df_new1 = pd.DataFrame()
weather_df_new1['combined'] = weather_df_new['station'] + weather_df_new['valid_new'].astype(str)
weather_df_new1['precip'] = weather_df_new['p01i']
weather_df_new1['vis'] = weather_df_new['vsby']
weather_df_new1['temp'] = weather_df_new['tmpf']
weather_df_new1['wind'] = weather_df_new['sknt']

In [34]:
weather_df_new1.head()

,combined,precip,vis,temp,wind
0,ATL2017-01-01 00:00:00,0.05,5.0,0.0,10.0
1,ATL2017-01-01 01:00:00,0.04,5.0,0.0,19.0
2,ATL2017-01-01 02:00:00,0.04,5.0,0.0,8.0
3,ATL2017-01-01 03:00:00,0.02,6.0,0.0,9.0
4,ATL2017-01-01 04:00:00,0.01,5.0,0.0,8.0


In [35]:
#suming up the precipitation for last five hours
sum_precip = []
for row in range(len(weather_df_new1.precip) - 5):
    rows_sum = sum(weather_df_new1.iloc[row:row + 5, 1].values)
    sum_precip.append(rows_sum)

In [36]:
sum_precip.insert(0,0)
sum_precip.insert(0,0)
sum_precip.insert(0,0)
sum_precip.insert(0,0)
sum_precip.insert(0,0)

In [37]:
#Averaging the visability for last five hours
ave_vis = []
for row in range(len(weather_df_new1.vis) - 5):
    rows_ave = np.mean(weather_df_new1.iloc[row:row + 5, 1].values)
    ave_vis.append(rows_ave)

In [38]:
ave_vis.insert(0,0)
ave_vis.insert(0,0)
ave_vis.insert(0,0)
ave_vis.insert(0,0)
ave_vis.insert(0,0)

In [39]:
weather_df_new1['ave_vis'] = ave_vis

In [40]:
weather_df_new1['precip_sum'] = sum_precip

In [41]:
weather_df_new1 = weather_df_new1.drop(columns=['precip','vis'])

In [42]:
weather_df_new1.head()

,combined,temp,wind,ave_vis,precip_sum
0,ATL2017-01-01 00:00:00,0.0,10.0,0.0,0.0
1,ATL2017-01-01 01:00:00,0.0,19.0,0.0,0.0
2,ATL2017-01-01 02:00:00,0.0,8.0,0.0,0.0
3,ATL2017-01-01 03:00:00,0.0,9.0,0.0,0.0
4,ATL2017-01-01 04:00:00,0.0,8.0,0.0,0.0


In [43]:
#do the same in the flight data 
df['combined'] =  df['Origin'] + df['FL_TIME_NEW'].astype(str)

In [44]:
df.head()

,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,Origin,OriginState,...,Dest,CRSDepTime,CRSArrTime,CRSElapsedTime,ArrDelay,Distance,DistanceGroup,FL_TIME,FL_TIME_NEW,combined
0,2,5,6,6,2017-05-06,WN,N7824A,4652,SJC,CA,...,SAN,650,810,80.0,2.0,417.0,2,2017-05-06 06:50:00,2017-05-06 06:00:00,SJC2017-05-06 06:00:00
1,2,5,6,6,2017-05-06,WN,N8522P,4971,SJC,CA,...,SAN,1710,1830,80.0,-7.0,417.0,2,2017-05-06 17:10:00,2017-05-06 17:00:00,SJC2017-05-06 17:00:00
2,2,5,6,6,2017-05-06,WN,N8617E,5113,SJC,CA,...,SAN,1100,1220,80.0,0.0,417.0,2,2017-05-06 11:00:00,2017-05-06 11:00:00,SJC2017-05-06 11:00:00
3,2,5,6,6,2017-05-06,WN,N450WN,5150,SJC,CA,...,SAN,2030,2150,80.0,1.0,417.0,2,2017-05-06 20:30:00,2017-05-06 20:00:00,SJC2017-05-06 20:00:00
4,2,5,6,6,2017-05-06,WN,N498WN,5711,SJC,CA,...,SAN,1325,1445,80.0,-2.0,417.0,2,2017-05-06 13:25:00,2017-05-06 13:00:00,SJC2017-05-06 13:00:00


In [45]:
copy_df = df.copy()

In [46]:
copy_df.head()

,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,Origin,OriginState,...,Dest,CRSDepTime,CRSArrTime,CRSElapsedTime,ArrDelay,Distance,DistanceGroup,FL_TIME,FL_TIME_NEW,combined
0,2,5,6,6,2017-05-06,WN,N7824A,4652,SJC,CA,...,SAN,650,810,80.0,2.0,417.0,2,2017-05-06 06:50:00,2017-05-06 06:00:00,SJC2017-05-06 06:00:00
1,2,5,6,6,2017-05-06,WN,N8522P,4971,SJC,CA,...,SAN,1710,1830,80.0,-7.0,417.0,2,2017-05-06 17:10:00,2017-05-06 17:00:00,SJC2017-05-06 17:00:00
2,2,5,6,6,2017-05-06,WN,N8617E,5113,SJC,CA,...,SAN,1100,1220,80.0,0.0,417.0,2,2017-05-06 11:00:00,2017-05-06 11:00:00,SJC2017-05-06 11:00:00
3,2,5,6,6,2017-05-06,WN,N450WN,5150,SJC,CA,...,SAN,2030,2150,80.0,1.0,417.0,2,2017-05-06 20:30:00,2017-05-06 20:00:00,SJC2017-05-06 20:00:00
4,2,5,6,6,2017-05-06,WN,N498WN,5711,SJC,CA,...,SAN,1325,1445,80.0,-2.0,417.0,2,2017-05-06 13:25:00,2017-05-06 13:00:00,SJC2017-05-06 13:00:00


In [47]:
weather_df_new1.head()

,combined,temp,wind,ave_vis,precip_sum
0,ATL2017-01-01 00:00:00,0.0,10.0,0.0,0.0
1,ATL2017-01-01 01:00:00,0.0,19.0,0.0,0.0
2,ATL2017-01-01 02:00:00,0.0,8.0,0.0,0.0
3,ATL2017-01-01 03:00:00,0.0,9.0,0.0,0.0
4,ATL2017-01-01 04:00:00,0.0,8.0,0.0,0.0


In [48]:
#mapping the weather data to the flight dataframe

In [49]:
weather_df_new1['zip'] = list(zip(weather_df_new1.temp, weather_df_new1.wind, weather_df_new1.ave_vis,
                                  weather_df_new1.precip_sum))

In [50]:
weather_df_new1.head()

,combined,temp,wind,ave_vis,precip_sum,zip
0,ATL2017-01-01 00:00:00,0.0,10.0,0.0,0.0,"(0.0, 10.0, 0.0, 0.0)"
1,ATL2017-01-01 01:00:00,0.0,19.0,0.0,0.0,"(0.0, 19.0, 0.0, 0.0)"
2,ATL2017-01-01 02:00:00,0.0,8.0,0.0,0.0,"(0.0, 8.0, 0.0, 0.0)"
3,ATL2017-01-01 03:00:00,0.0,9.0,0.0,0.0,"(0.0, 9.0, 0.0, 0.0)"
4,ATL2017-01-01 04:00:00,0.0,8.0,0.0,0.0,"(0.0, 8.0, 0.0, 0.0)"


In [51]:
look_up = weather_df_new1.copy().drop(columns=['temp','wind','ave_vis','precip_sum'])

In [52]:
look_up = look_up.set_index('combined')

In [53]:
look_up.head()

,zip
combined,
ATL2017-01-01 00:00:00,"(0.0, 10.0, 0.0, 0.0)"
ATL2017-01-01 01:00:00,"(0.0, 19.0, 0.0, 0.0)"
ATL2017-01-01 02:00:00,"(0.0, 8.0, 0.0, 0.0)"
ATL2017-01-01 03:00:00,"(0.0, 9.0, 0.0, 0.0)"
ATL2017-01-01 04:00:00,"(0.0, 8.0, 0.0, 0.0)"


In [54]:
weather_dict = look_up.to_dict()['zip']

In [55]:
copy_df['zipped'] = copy_df.combined.map(weather_dict)

In [56]:
copy_df = copy_df.fillna(0)

In [57]:
copy_df['zipped'] = copy_df.zipped.apply(lambda x: (0,0,0,0) if x==0 else x)

In [58]:
 copy_df[['tempF','wind','ave_vis','precip_sum']] = pd.DataFrame(copy_df['zipped'].tolist(), index=df.index)                                                                                                                       

In [59]:
copy_df.head()

,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,Origin,OriginState,...,Distance,DistanceGroup,FL_TIME,FL_TIME_NEW,combined,zipped,tempF,wind,ave_vis,precip_sum
0,2,5,6,6,2017-05-06,WN,N7824A,4652,SJC,CA,...,417.0,2,2017-05-06 06:50:00,2017-05-06 06:00:00,SJC2017-05-06 06:00:00,"(0, 0, 0, 0)",0.0,0.0,0.0,0.0
1,2,5,6,6,2017-05-06,WN,N8522P,4971,SJC,CA,...,417.0,2,2017-05-06 17:10:00,2017-05-06 17:00:00,SJC2017-05-06 17:00:00,"(0, 0, 0, 0)",0.0,0.0,0.0,0.0
2,2,5,6,6,2017-05-06,WN,N8617E,5113,SJC,CA,...,417.0,2,2017-05-06 11:00:00,2017-05-06 11:00:00,SJC2017-05-06 11:00:00,"(0, 0, 0, 0)",0.0,0.0,0.0,0.0
3,2,5,6,6,2017-05-06,WN,N450WN,5150,SJC,CA,...,417.0,2,2017-05-06 20:30:00,2017-05-06 20:00:00,SJC2017-05-06 20:00:00,"(0, 0, 0, 0)",0.0,0.0,0.0,0.0
4,2,5,6,6,2017-05-06,WN,N498WN,5711,SJC,CA,...,417.0,2,2017-05-06 13:25:00,2017-05-06 13:00:00,SJC2017-05-06 13:00:00,"(0, 0, 0, 0)",0.0,0.0,0.0,0.0


In [78]:
#export the dataframe as a csv to model
copy_df.to_csv(f'{flight_path}2017NEW.csv', index = False, encoding = 'utf-8')